In [1]:
import dagshub
dagshub.init(repo_owner='sahilbhardwaj23', repo_name='Youtube-comment-Sentiment-Analysis', mlflow=True)

Accessing as sahilbhardwaj23

Initialized MLflow to track repo "sahilbhardwaj23/Youtube-comment-Sentiment-Analysis"

Repository sahilbhardwaj23/Youtube-comment-Sentiment-Analysis initialized!

In [2]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/sahilbhardwaj23/Youtube-comment-Sentiment-Analysis.mlflow")

In [3]:
from mlflow.tracking import MlflowClient
import mlflow

In [4]:
def set_or_create_experiment(experiment_name):
    client = MlflowClient()
    experiment = client.get_experiment_by_name(experiment_name)

    # Check if the experiment exists and is active
    if experiment:
        if experiment.lifecycle_stage == "active":
            print(f"Using existing experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            mlflow.set_experiment(experiment_name)
        else:
            # Restore if it's deleted
            print(f"Restoring deleted experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            client.restore_experiment(experiment.experiment_id)
            mlflow.set_experiment(experiment_name)
    else:
        # Create a new experiment if it doesn't exist
        print(f"Creating new experiment '{experiment_name}'")
        mlflow.set_experiment(experiment_name)



In [5]:
# Set or create an experiment
set_or_create_experiment("ML Algos with HP Tuning")

Using existing experiment 'ML Algos with HP Tuning' (ID: 7)


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


e:\Projects\Youtube Comment Analysis\Youtube commnent Plugin\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [8]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2024-11-03 10:32:13,254] A new study created in memory with name: no-name-02ba4eca-e4dd-4017-a912-c34b4a45e909
[I 2024-11-03 10:32:13,536] Trial 0 finished with value: 0.3333333333333333 and parameters: {'C': 0.0004449409887974894, 'penalty': 'l1'}. Best is trial 0 with value: 0.3333333333333333.
[I 2024-11-03 10:32:14,663] Trial 1 finished with value: 0.7687592475163814 and parameters: {'C': 2.3033269806396994, 'penalty': 'l2'}. Best is trial 1 with value: 0.7687592475163814.
[I 2024-11-03 10:32:14,779] Trial 2 finished with value: 0.3333333333333333 and parameters: {'C': 0.00018213932551308868, 'penalty': 'l1'}. Best is trial 1 with value: 0.7687592475163814.
[I 2024-11-03 10:32:15,272] Trial 3 finished with value: 0.779433523567956 and parameters: {'C': 1.8674199396221651, 'penalty': 'l1'}. Best is trial 3 with value: 0.779433523567956.
[I 2024-11-03 10:32:15,838] Trial 4 finished with value: 0.7810188120904671 and parameters: {'C': 6.053843606969535, 'penalty': 'l1'}. Best is tr